
# Algorithmic Trading Machine Learning Project

This is a machine learning project that will take in S&P 500 stock price data and use an unsupervised learning trading strategy. This is strictly for educational purposes and is to help me learn machine learning.

1. Install packages.

In [31]:
# Packages installed:
# pandas
# pandas_ta
# numpy
# matplotlib
# statsmodels
# pandas_datareader
# datetime
# yfinance
# sklearn
# PyPortfolioOpt

!pip install pandas pandas_ta numpy matplotlib statsmodels pandas_datareader datetime yfinance sklearn PyPortfolioOpt -q


You should consider upgrading via the 'c:\users\kyle\onedrive\desktop\project folders\algoirthmic trading\my_virtual_env\scripts\python.exe -m pip install --upgrade pip' command.


2. Download S&P 500 data

In [32]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

# Read the S&P 500 list of companies 
# pd.read_html is used for reading HTML tables from a webpage
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# Remove the dot from the symbol names
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# Get the list of symbols
symbols_list = sp500['Symbol'].unique().tolist()
symbols_list

# Define the start and end dates
end_date = '2023-09-27'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365 * 8)

# Download the data
df = yf.download(tickers = symbols_list, 
                 start=start_date, 
                 end=end_date).stack()

df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()


[*********************100%%**********************]  503 of 503 completed

3 Failed downloads:
['VLTO', 'SOLV', 'GEV']: YFChartError("%ticker%: Data doesn't exist for startDate = 1443499200, endDate = 1695787200")


2. Technical indicators for all the stocks

In [33]:
# Garman-Klass volatility
# Used to estimate volitility of an asset
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) **2/2 - ((2 * np.log(2) - 1) * (np.log(df['adj close']) - np.log(df['open'])) ** 2))

# RSI
# Relative Strength Index used for overbought and oversold areas
df['rsi'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.rsi(close = x, length = 20))

# Bolinger Bands
# Consists of 3 bands, the lower, middle and upper band
df['bb_low'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:, 0])
df['bb_med'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:, 1])
df['bb_high'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:, 2])

# ATR
# Average True Range used to measure volatility
def compute_atr(stock_data):
    atr = pandas_ta.atr(high = stock_data['high'],
                        low = stock_data['low'],
                        close = stock_data['adj close'],
                        length = 14)
    return atr.sub(atr.mean()).div(atr.std())
df['atr'] = df.groupby(level = 1, group_keys = False).apply(compute_atr)

# MACD
# Identify trend direction and momentum
def compute_macd(close):
    macd = pandas_ta.macd(close = close, length = 20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level = 1, group_keys = False)['adj close'].apply(compute_macd)

# Volume
# The number of shares traded
df['dollar_vol'] = (df['adj close'] * df['volume']) / 1e6
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.425230   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651127   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334900   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478706   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736305   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211006  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547714  176.869995  178.449997  176.270004  176.580002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.001082        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005712        NaN       NaN   
           ABBV     12842800.0         -0.054655        NaN       NaN   
           ABT      12287500.0         -0.009402        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146729  4.485761   
           YUM       1500600.0         -0.000051  36.057170  4.811707   
           ZBH       3610500.0          0.000022  31.893235  4.745884   
           ZBRA       355400.0          0.000133  29.494977  5.400991   
           ZTS       1463200.0          0.000062  42.623488  5.156108   

Price                bb_med   bb_high       atr      macd   dollar_vol  
date       ticker                                                       
2015-09-29 A            NaN       NaN       NaN       NaN    70.782188  
           AAL          NaN       NaN       NaN       NaN   279.420098  
           AAPL         NaN       NaN       NaN       NaN  7234.159131  
           ABBV         NaN       NaN       NaN       NaN   466.641852  
           ABT          NaN       NaN       NaN       NaN   411.369604  
...                     ...       ...       ...       ...          ...  
2023-09-26 XYL     4.567684  4.649607 -2.967042 -2.159188   117.344890  
           YUM     4.841672  4.871637 -2.813230 -1.363696   183.389836  
           ZBH     4.785551  4.825217 -2.109952 -0.881067   402.696470  
           ZBRA    5.539167  5.677342 -0.057389 -1.600791    79.595386  
           ZTS     5.214922  5.273735 -0.720920 -1.188278   256.861415  

[987967 rows x 14 columns]

3. Filter top 150 most liquid

In [34]:
# Define the list of columns to keep
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_vol', 'volume', 'open', 'high', 'low', 'close']]

data = pd.concat([df.unstack('ticker')['dollar_vol'].resample('M').mean().stack('ticker').to_frame('dollar_vol'),
df.unstack()[last_cols].resample('M').last().stack('ticker')], axis = 1).dropna()

data

dollar_vol   adj close       atr   bb_high    bb_low  \
date       ticker                                                          
2015-11-30 A        135.740904   38.950890 -0.948795  3.689080  3.544191   
           AAL      287.915810   39.429928  1.473834  3.827636  3.672028   
           AAPL    4023.983936   26.854137 -0.326058  3.368302  3.281679   
           ABBV     337.563930   40.393524  0.572008  3.823191  3.726695   
           ABT      211.659041   38.293575  0.553866  3.699763  3.656055   
...                        ...         ...       ...       ...       ...   
2023-09-30 OTIS     154.361755   78.356499 -2.269284  4.460712  4.370137   
           ABNB    1633.500725  132.279999 -1.006939  5.024801  4.857047   
           CEG      196.670368  107.862022 -0.705545  4.732493  4.652147   
           GEHC     212.275852   66.130219 -0.905988  4.270508  4.155436   
           KVUE     670.804278   20.006195 -2.650791  3.164168  3.014275   

                     bb_med  garman_klass_vol      macd        rsi  
date       ticker                                                   
2015-11-30 A       3.616636         -0.002098  0.567157  73.421427  
           AAL     3.749832         -0.000966 -0.418772  40.718913  
           AAPL    3.324990         -0.003307 -0.142789  55.537370  
           ABBV    3.774943         -0.059549  0.145677  49.376840  
           ABT     3.677909         -0.011216  0.335557  56.962590  
...                     ...               ...       ...        ...  
2023-09-30 OTIS    4.415425         -0.000097 -1.534536  33.116229  
           ABNB    4.940924          0.000213 -0.037854  44.494127  
           CEG     4.692320          0.000131  0.366876  55.245468  
           GEHC    4.212972          0.000185 -1.116463  40.922338  
           KVUE    3.089221         -0.000159 -1.435620  35.706355  

[46648 rows x 9 columns]

In [35]:
# Rolling average 5 year
data['dollar_vol'] = (data.loc[:,'dollar_vol'].unstack('ticker').rolling(5*12, min_periods = 12).mean().stack())

# Rank the stocks by dollar volume
data['dollar_vol_rank'] = (data.groupby('date')['dollar_vol'].rank(ascending=False))

# Keep only the top 150 stocks by dollar volume
data = data[data['dollar_vol_rank']<150].drop(['dollar_vol', 'dollar_vol_rank'], axis=1)

data

adj close       atr   bb_high    bb_low    bb_med  \
date       ticker                                                       
2016-10-31 AAL      39.134335  1.161163  3.706314  3.604673  3.655494   
           AAPL     26.212473 -0.963372  3.351997  3.294237  3.323117   
           ABBV     40.245121  0.257529  3.861774  3.753446  3.807610   
           ABT      34.293453 -0.483970  3.655580  3.554632  3.605106   
           ACN     103.569618  0.222959  4.660617  4.637342  4.648980   
...                       ...       ...       ...       ...       ...   
2023-09-30 XOM     113.372101 -1.345888  4.767283  4.687091  4.727187   
           MRNA     98.120003 -0.529511  4.788149  4.582514  4.685332   
           UBER     44.270000 -0.746098  3.917801  3.806654  3.862227   
           CRWD    160.479996 -0.744862  5.181204  5.026187  5.103696   
           ABNB    132.279999 -1.006939  5.024801  4.857047  4.940924   

                   garman_klass_vol      macd        rsi  
date       ticker                                         
2016-10-31 AAL            -0.000176  1.131596  62.203576  
           AAPL           -0.002468 -0.195978  49.891067  
           ABBV           -0.046691 -0.760594  27.477760  
           ABT            -0.007492 -0.650888  38.008880  
           ACN            -0.004643 -0.135457  53.823625  
...                             ...       ...        ...  
2023-09-30 XOM            -0.000065  1.400624  59.440184  
           MRNA            0.000146 -0.376899  38.747314  
           UBER            0.000441 -0.133973  45.005268  
           CRWD            0.000144  0.245950  51.534803  
           ABNB            0.000213 -0.037854  44.494127  

[12516 rows x 8 columns]

4. Calculate monthly returns for different times


In [36]:
# Calculate the returns function
def calculate_returns(df):
    # 95%
    outlier_cutoff = 0.005
    # Define the time frames
    lags = [1,2,3,6,9,12]

    # Calculate the returns
    for lag in lags:
        df[f'return_{lag}m'] = df['adj close'].pct_change(lag).pipe(lambda x: x.clip(lower = x.quantile(outlier_cutoff), upper = x.quantile(1-outlier_cutoff))).add(1).pow(1/lag).sub(1)
    return df
data = data.groupby(level = 1, group_keys = False).apply(calculate_returns).dropna()
data



adj close       atr   bb_high    bb_low    bb_med  \
date       ticker                                                       
2017-10-31 AAL      45.534168  1.125077  3.994389  3.849110  3.921750   
           AAPL     39.713894 -0.532950  3.688475  3.594730  3.641603   
           ABBV     67.491142  1.577038  4.289423  4.196702  4.243063   
           ABT      48.493374 -0.019444  3.939707  3.892568  3.916137   
           ACN     129.399689  0.250124  4.882034  4.802675  4.842354   
...                       ...       ...       ...       ...       ...   
2023-09-30 XOM     113.372101 -1.345888  4.767283  4.687091  4.727187   
           MRNA     98.120003 -0.529511  4.788149  4.582514  4.685332   
           UBER     44.270000 -0.746098  3.917801  3.806654  3.862227   
           CRWD    160.479996 -0.744862  5.181204  5.026187  5.103696   
           ABNB    132.279999 -1.006939  5.024801  4.857047  4.940924   

                   garman_klass_vol      macd        rsi  return_1m  \
date       ticker                                                     
2017-10-31 AAL            -0.000363 -0.018697  41.051781  -0.014108   
           AAPL           -0.001055 -0.039276  69.196651   0.096807   
           ABBV           -0.034008  0.473815  55.247847   0.022728   
           ABT            -0.005190  0.276132  53.844909   0.021276   
           ACN            -0.003925  0.352342  69.365138   0.064180   
...                             ...       ...        ...        ...   
2023-09-30 XOM            -0.000065  1.400624  59.440184   0.046947   
           MRNA            0.000146 -0.376899  38.747314  -0.132219   
           UBER            0.000441 -0.133973  45.005268  -0.062672   
           CRWD            0.000144  0.245950  51.534803  -0.015641   
           ABNB            0.000213 -0.037854  44.494127   0.005549   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2017-10-31 AAL      0.022981  -0.023860   0.016495   0.007008    0.012702  
           AAPL     0.015249   0.044955   0.028875   0.038941    0.035228  
           ABBV     0.098590   0.091379   0.056495   0.047273    0.044026  
           ABT      0.034308   0.034801   0.038672   0.031320    0.029294  
           ACN      0.048454   0.037202   0.028692   0.027398    0.018728  
...                      ...        ...        ...        ...         ...  
2023-09-30 XOM      0.046139   0.030496   0.012838   0.008747    0.027037  
           MRNA    -0.086803  -0.068763  -0.071952  -0.064976   -0.015431  
           UBER    -0.053920   0.008422   0.057244   0.066838    0.043691  
           CRWD    -0.003656   0.029981   0.026391   0.047942   -0.002216  
           ABNB    -0.067704   0.010603   0.010289   0.049124    0.019401  

[10344 rows x 14 columns]

5. Download Fama-French factors and calculate rolling factor betas.


In [37]:
# Get the Fama-French factors data from library
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
               'famafrench',
               start = '2010',)[0].drop('RF', axis = 1)

# Convert the index to datetime
factor_data.index = factor_data.index.to_timestamp()
factor_data.index.name = 'date'

# Get last value of each month and divide by 100
factor_data = factor_data.resample('M').last().div(100)

# Join the factor with our data we pulled from S&P 500
factor_data = factor_data.join(data['return_1m']).sort_index()
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2017-10-31 AAL     0.0225 -0.0194  0.0020  0.0093 -0.0325  -0.014108
           AAPL    0.0225 -0.0194  0.0020  0.0093 -0.0325   0.096807
           ABBV    0.0225 -0.0194  0.0020  0.0093 -0.0325   0.022728
           ABT     0.0225 -0.0194  0.0020  0.0093 -0.0325   0.021276
           ACN     0.0225 -0.0194  0.0020  0.0093 -0.0325   0.064180
...                   ...     ...     ...     ...     ...        ...
2023-09-30 VRTX   -0.0524 -0.0181  0.0151  0.0187 -0.0082   0.009617
           VZ     -0.0524 -0.0181  0.0151  0.0187 -0.0082  -0.056890
           WFC    -0.0524 -0.0181  0.0151  0.0187 -0.0082  -0.015500
           WMT    -0.0524 -0.0181  0.0151  0.0187 -0.0082  -0.000677
           XOM    -0.0524 -0.0181  0.0151  0.0187 -0.0082   0.046947

[10344 rows x 6 columns]

In [38]:
# Filter stocks without 10 months of data
# We need at least 10 months of data to calculate the factor or else the model will break
observations = factor_data.groupby(level = 1).size()
valid_stocks = observations[observations >= 10]
factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2017-10-31 AAL     0.0225 -0.0194  0.0020  0.0093 -0.0325  -0.014108
           AAPL    0.0225 -0.0194  0.0020  0.0093 -0.0325   0.096807
           ABBV    0.0225 -0.0194  0.0020  0.0093 -0.0325   0.022728
           ABT     0.0225 -0.0194  0.0020  0.0093 -0.0325   0.021276
           ACN     0.0225 -0.0194  0.0020  0.0093 -0.0325   0.064180
...                   ...     ...     ...     ...     ...        ...
2023-09-30 VRTX   -0.0524 -0.0181  0.0151  0.0187 -0.0082   0.009617
           VZ     -0.0524 -0.0181  0.0151  0.0187 -0.0082  -0.056890
           WFC    -0.0524 -0.0181  0.0151  0.0187 -0.0082  -0.015500
           WMT    -0.0524 -0.0181  0.0151  0.0187 -0.0082  -0.000677
           XOM    -0.0524 -0.0181  0.0151  0.0187 -0.0082   0.046947

[10314 rows x 6 columns]

In [39]:
# calculate rolling betas.
# 24 month window size
betas = factor_data.groupby(level = 1,
                    group_keys = False).apply(lambda x: RollingOLS(endog = x['return_1m'], exog = sm.add_constant( x.drop('return_1m', axis = 1)),
                                              window = min(24, x.shape[0]),
                                              min_nobs = len(x.columns) + 1)
.fit()
.params
.drop('const', axis = 1))

betas

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2017-10-31 AAL          NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACN          NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2023-09-30 VRTX    0.456835 -0.444629 -0.314191 -0.077989  0.802008
           VZ      0.332723 -0.166037  0.265928  0.311103  0.108624
           WFC     1.120622  0.297484  2.062607 -0.441341 -1.519516
           WMT     0.700775 -0.313572 -0.413680 -0.141574  0.508837
           XOM     0.983098 -1.094427  1.756406 -0.640016 -0.368887

[10314 rows x 5 columns]

In [44]:
# Fix null values
# Total of 18 features to be used for the machine learning model
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
data = data.join(betas.groupby('ticker').shift())
data.loc[:, factors] = data.groupby('ticker', group_keys = False)[factors].apply(lambda x: x.fillna(x.mean()))
data = data.drop('adj close', axis = 1)
data = data.dropna()
data.info()

# The data is ready for the machine learning model.
# The model will predict which stocks should be chosen to maximize gains for the next month.

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10094 entries, (Timestamp('2017-10-31 00:00:00'), 'AAL') to (Timestamp('2023-09-30 00:00:00'), 'CRWD')
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   atr               10094 non-null  float64
 1   bb_high           10094 non-null  float64
 2   bb_low            10094 non-null  float64
 3   bb_med            10094 non-null  float64
 4   garman_klass_vol  10094 non-null  float64
 5   macd              10094 non-null  float64
 6   rsi               10094 non-null  float64
 7   return_1m         10094 non-null  float64
 8   return_2m         10094 non-null  float64
 9   return_3m         10094 non-null  float64
 10  return_6m         10094 non-null  float64
 11  return_9m         10094 non-null  float64
 12  return_12m        10094 non-null  float64
 13  Mkt-RF            10094 non-null  float64
 14  SMB               10094 non-null  float64
 15  HML       